# PASOS PARA LA DESCARGA DEL DATASET Y PREPARACIÓN

Crear el ambiente virtual para que incluso si muevo archivos funcione:

En esta hoja voy a detallar todos los procesos que me han funcionado para poder descargar el dataset, transformar  y dejar todo listo para correr el modelo:

# 0) Conectar con Drive

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1) Descargar el dataset con las anotaciones desde Roboflow

cuando descargo versión, lo hago produciendo versión con auto-oriented, nada más, no tiling no resize (esto la haré luego yo)

In [ ]:
#if not installed, you need to install roboflow in terminal  : pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="9eeIO2PsJSso62o9")

project = rf.workspace("seals-cqpqc").project("aws-qgxmp")          # nombre del workspace y del projecto
dataset = project.version(10).download("YOLOV8")                     # "6" ES la versión que descargué , adaptala a la tuya. YOLOV8 es el formato


#2) Transformar las anotaciones a formato YOLO (es decir, bounding box)

esto es porque al decargar anotaciones, yo tengo poligonos y bounding boxes, quiero que todas sean bounding boxed que es lo que el modelo acepta

Haces carpeta a carpeta (train- validation-test)

otra opción es: Use a framework that accepts mixed annotations without modification (e.g., PyTorch with COCO JSON format).

In [ ]:
import os

def polygon_to_bbox(coords):
    """
    Convierte una lista de coordenadas de un polígono (x1, y1, x2, y2, ..., xN, yN)
    a una bounding box en formato YOLO: (x_center, y_center, width, height),
    asumiendo que las coordenadas están normalizadas [0,1].
    """
    xs = coords[0::2]  # extrae todos los valores de x
    ys = coords[1::2]  # extrae todos los valores de y

    x_min = min(xs)
    x_max = max(xs)
    y_min = min(ys)
    y_max = max(ys)

    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min

    return x_center, y_center, width, height

def process_label_file(src_file, dst_file):
    """
    Lee un archivo de anotaciones en formato YOLO (líneas con la clase seguida de coordenadas).
    Si la línea contiene 4 números (bounding box), la deja igual.
    Si contiene más de 4 (asumiendo un polígono), lo convierte a bounding box.
    Escribe el resultado en un nuevo archivo.
    """
    with open(src_file, 'r') as f_in:
        lines = f_in.readlines()

    new_lines = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        cls = parts[0]  # la clase (por ejemplo, "0", "1", "2", etc.)       ##clases indicadas en data.yaml
        coords = list(map(float, parts[1:]))

        if len(coords) == 4:
            # Ya es bounding box, se deja igual.
            new_line = f"{cls} {coords[0]:.16f} {coords[1]:.16f} {coords[2]:.16f} {coords[3]:.16f}"
        else:
            # Es un polígono; se convierte a bounding box.
            x_center, y_center, width, height = polygon_to_bbox(coords)
            new_line = f"{cls} {x_center:.16f} {y_center:.16f} {width:.16f} {height:.16f}"      #puedes modificar el número de decimales(16, o los que quieras)
        new_lines.append(new_line)

    with open(dst_file, 'w') as f_out:
        for new_line in new_lines:
            f_out.write(new_line + "\n")

    print(f"Procesado: {os.path.basename(src_file)} -> {os.path.basename(dst_file)}")

def process_all_labels(src_labels_dir, dst_labels_dir):
    """
    Procesa todos los archivos .txt en src_labels_dir y guarda los nuevos archivos
    (con anotaciones convertidas a bounding boxes) en dst_labels_dir.
    """
    os.makedirs(dst_labels_dir, exist_ok=True)
    for filename in os.listdir(src_labels_dir):
        if filename.endswith(".txt"):
            src_path = os.path.join(src_labels_dir, filename)
            dst_path = os.path.join(dst_labels_dir, filename)
            process_label_file(src_path, dst_path)

if __name__ == "__main__":
    ###############################################################################################33
    # Rutas de entrada y salida: modifica estas rutas a las de tu sistema
    src_labels_dir = "E:/fotos_modelo_5000_fina/valid/labels"
    dst_labels_dir = "E:/fotos_modelo_5000_fina/valid/valid_bbconverted_labels"

    ################################################################################################
    process_all_labels(src_labels_dir, dst_labels_dir)

    print("¡Conversión completada!")

#3) Comprobar que esas anotaciones que ha hecho, corresponden con las imágenes y están bien hechas

SOLO UNA IMAGEN

Saldrá la imagen con las anotaciones.

In [ ]:
#import supervision, cv2 and numpy si lo necesitas en terminal, como antes: pip install supervision, cv2, numpy
!pip install supervision
!pip install cv2
!pip install numpy
import supervision as sv
import cv2
import numpy as np

# Load the image
image = cv2.imread("/content/drive/MyDrive/fotos_pruebas/resultados_bgr2/dataset_aug11/images/DSC02325_rotate.jpg")

# Load the labels from the txt file
with open("/content/drive/MyDrive/fotos_pruebas/resultados_bgr2/dataset_aug11/labels/DSC02325_rotate.txt", "r") as f:
    lines = f.readlines()

# Parse the labels
xyxy = []
confidence = []
class_id = []
for line in lines:
    class_id_, x_center, y_center, width, height = map(float, line.strip().split())
    x1 = (x_center - width/2) * image.shape[1]
    y1 = (y_center - height/2) * image.shape[0]
    x2 = (x_center + width/2) * image.shape[1]
    y2 = (y_center + height/2) * image.shape[0]
    xyxy.append([x1, y1, x2, y2])
    confidence.append(1.0)  # Assuming confidence of 1.0 for all detections
    class_id.append(class_id_)

# Convert to supervision Detections object
detections = sv.Detections(
    xyxy=np.array(xyxy),
    confidence=np.array(confidence),
    class_id=np.array(class_id).astype(int)
)

# Create annotators
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Annotate the image
annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=[f"Class {int(class_id)}" for class_id in detections.class_id])

# Display the annotated image
sv.plot_image(image=annotated_frame, size=(16, 16))

#4. Tiling en carpetas grandes


**    IMPORTANTE     **

Aquí he de cambiar el tamaño de tiles que yo quierO

4.1 Generando todo en una carpeta    
(también está el código para hacer subcarpetas de cada imagen con tiles y con txt)
para cada imagen , partiendo de imagen y txt, crea tiles que van asociadas a txt, por lo tanto defines rutas entrada, salida y tamaño tiles

In [ ]:
import cv2
import os
import numpy as np

def load_annotations(label_path, image_width, image_height):
    """
    Lee el archivo de anotaciones en formato YOLO (bounding boxes o polígonos)
    y devuelve una lista de anotaciones en coordenadas absolutas (en píxeles):
    (cls, x1, y1, x2, y2)
    """
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls = parts[0]
            coords = list(map(float, parts[1:]))

            if len(coords) == 4:
                # Ya es bounding box (normalizada: x_center, y_center, w, h)
                x_center, y_center, w, h = coords
                abs_x_center = x_center * image_width
                abs_y_center = y_center * image_height
                abs_w = w * image_width
                abs_h = h * image_height
                x1 = abs_x_center - abs_w/2
                y1 = abs_y_center - abs_h/2
                x2 = abs_x_center + abs_w/2
                y2 = abs_y_center + abs_h/2
            else:
                # Es un polígono: convertir a bounding box envolvente
                xs = coords[0::2]
                ys = coords[1::2]
                x1 = min(xs) * image_width
                x2 = max(xs) * image_width
                y1 = min(ys) * image_height
                y2 = max(ys) * image_height

            annotations.append((cls, x1, y1, x2, y2))
    return annotations

def adjust_annotation_for_tile(annotation, tile_x, tile_y, tile_size):
    """
    Dada una anotación en coordenadas absolutas (x1, y1, x2, y2) de la imagen original,
    y el offset (tile_x, tile_y) del tile, calcula la intersección entre la caja y el tile.
    Devuelve la bounding box transformada a coordenadas locales del tile (normalizadas)
    o None si no hay intersección.
    """
    cls, x1, y1, x2, y2 = annotation
    tile_x1, tile_y1 = tile_x, tile_y
    tile_x2, tile_y2 = tile_x + tile_size, tile_y + tile_size

    # Calcular la intersección
    inter_x1 = max(x1, tile_x1)
    inter_y1 = max(y1, tile_y1)
    inter_x2 = min(x2, tile_x2)
    inter_y2 = min(y2, tile_y2)

    if inter_x2 <= inter_x1 or inter_y2 <= inter_y1:
        return None  # No hay intersección

    # Convertir a coordenadas locales del tile
    local_x1 = inter_x1 - tile_x
    local_y1 = inter_y1 - tile_y
    local_x2 = inter_x2 - tile_x
    local_y2 = inter_y2 - tile_y

    box_w = local_x2 - local_x1
    box_h = local_y2 - local_y1
    x_center = local_x1 + box_w / 2.0
    y_center = local_y1 + box_h / 2.0
    norm_x_center = x_center / tile_size
    norm_y_center = y_center / tile_size
    norm_w = box_w / tile_size
    norm_h = box_h / tile_size

    return (cls, norm_x_center, norm_y_center, norm_w, norm_h)

def process_image_tiling(image_path, label_path, tile_size, output_images_dir, output_labels_dir, overlap=0):
    """
    Realiza el tiling de una imagen y ajusta sus anotaciones.
    Guarda cada tile en output_images_dir y su etiqueta correspondiente en output_labels_dir.
    'overlap' (en píxeles) permite solapar los tiles.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error al cargar {image_path}")
        return
    orig_h, orig_w, _ = image.shape

    annotations = load_annotations(label_path, orig_w, orig_h)

    tile_idx = 0
    # Iterar en el eje Y y X con el paso (tile_size - overlap)
    step = tile_size - overlap
    for y in range(0, orig_h, step):
        for x in range(0, orig_w, step):
            # Extraer tile de la imagen (puede ser más pequeño en bordes)
            tile = image[y:min(y+tile_size, orig_h), x:min(x+tile_size, orig_w)]
            tile_h, tile_w, _ = tile.shape
            # Si el tile es menor, aplicar padding para que sea de tamaño tile_size x tile_size
            if tile_h < tile_size or tile_w < tile_size:
                padded_tile = np.zeros((tile_size, tile_size, 3), dtype=tile.dtype)
                padded_tile[0:tile_h, 0:tile_w] = tile
                tile = padded_tile

            # Nombrar el tile usando el nombre original de la imagen y un índice
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            tile_filename = f"{base_name}_tile_{tile_idx}.jpg"
            cv2.imwrite(os.path.join(output_images_dir, tile_filename), tile)

            # Ajustar las anotaciones para el tile
            tile_annotations = []
            for ann in annotations:
                adj_ann = adjust_annotation_for_tile(ann, x, y, tile_size)
                if adj_ann is not None:
                    tile_annotations.append(adj_ann)

            # Guardar la anotación del tile
            label_filename = f"{base_name}_tile_{tile_idx}.txt"
            with open(os.path.join(output_labels_dir, label_filename), "w") as f:
                for ann in tile_annotations:
                    cls, xc, yc, w, h = ann
                    f.write(f"{cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

            tile_idx += 1
    print(f"Procesada {os.path.basename(image_path)}: {tile_idx} tiles generados.")

def process_dataset(source_images_dir, source_labels_dir, output_images_dir, output_labels_dir, tile_size=960, overlap=0):
    """
    Procesa todas las imágenes en source_images_dir, buscando para cada una su .txt
    en source_labels_dir. Guarda todos los tiles en output_images_dir y
    todas las anotaciones en output_labels_dir.
    """
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img_file in image_files:
        image_path = os.path.join(source_images_dir, img_file)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        label_path = os.path.join(source_labels_dir, label_file)
        if not os.path.exists(label_path):
            print(f"Advertencia: No se encontró etiqueta para {img_file}. Saltando...")
            continue
        process_image_tiling(image_path, label_path, tile_size, output_images_dir, output_labels_dir, overlap=overlap)

if __name__ == "__main__":
    # Rutas de ejemplo: ajusta estas rutas a las de tu sistema
    source_images_dir = "E:/fotos_modelo_5000_fina/test/images"
    source_labels_dir = "E:/fotos_modelo_5000_fina/test/test_bbconverted_labels"


    # Directorios de salida unificados para tiles e anotaciones
    output_images_dir = "C:/Users/CABALLERO DE ROHAN/Desktop/new_model_640_labels/test/images"
    output_labels_dir = "C:/Users/CABALLERO DE ROHAN/Desktop/new_model_640_labels/test/labels"


####################################################################################################################################################
####################################################################################################################################################


    # Parámetros: tile size y overlap (si quieres solapamiento, por ejemplo, 50 píxeles)

    tile_size = 640
    overlap = 50  # Puedes ajustar o poner 0 si no quieres solapamiento.  ## mejor solapamiento?  --> chatgp:  -->Sí, solapar (overlap) los tiles es una práctica común. Cuando un objeto (o una caja) cae justo en el borde de un tile, si no hay solapamiento existe el riesgo de que la caja se corte y se pierda parte de la información. Con un solapamiento, ese objeto puede aparecer (completamente o en mayor medida) en el tile adyacente. Esto mejora la probabilidad de que el objeto se detecte correctamente.


####################################################################################################################################################
####################################################################################################################################################

    process_dataset(source_images_dir, source_labels_dir, output_images_dir, output_labels_dir, tile_size=tile_size, overlap=overlap)
    print("Tiling de todas las imágenes completado.")